### Taxonomy Structure
| Parent                     | Child List                                                                    |
|----------------------------|-------------------------------------------------------------------------------|
| Lung Opacity               | Pneumonia ,  Atelectasis , Consolidation , Lung Lesion , Edema , Infiltration |
| Enlarged Cardiomediastinum | Cardiomegaly                                                                  |
| Infiltration               | Consolidation                                                                 |

In [3]:
%load_ext autoreload
%autoreload 2
from main.aims.aim1_1_taxonomy.utils_taxonomy import *

# Run

In [ ]:
# System.list_physical_devices

In [ ]:
approach = 'loss' # 'logit'
aim1_1 = AIM1_1_TorchXrayVision.run_full_experiment(approach=approach)

# Detailed

### Loading train/test data as well as the pre-trained model

In [ ]:
# Getting the user arguments
config = reading_user_input_arguments(jupyter=True)

# Initializing the class
FE = AIM1_1_TorchXrayVision(config=config)

In [ ]:
FE.train, FE.test, FE.model = AIM1_1_TorchXrayVision.load_data_and_model(FE.config)

In [ ]:
# labels = FE.train.labels

# n_classes = len(labels.unique)
# total_requested_n_samples = config.n_batches_to_process * config.batch_size

# # n_samples_per_class = labels.totals.apply(lambda row: min(row.min(), total_requested_n_samples / n_classes) ).to_frame('n_samples_per_class').replace(np.nan, '').T

# class_ratio = labels.totals.apply(lambda row: row[0]/row[1]).replace(np.nan, '').T

# class_ratio

### Measuring the ORIGINAL metrics (predictions and losses, thresholds, aucs, etc.)

In [ ]:
param = {key: getattr(FE, key) for key in ['model', 'config']}
FE.train = CalculateOriginalFindings.get_updated_data(data=FE.train, **param)
FE.test  = CalculateOriginalFindings.get_updated_data(data=FE.test , **param)

### Calculating the hyperparameters

In [ ]:
FE.hyperparameters = HyperParameterTuning.get_updated_data(data=FE.train, **param)

In [ ]:
FE.get_metric(metric='AUC', data_mode='train')

In [ ]:
FE.hyperparameters['ROC']

### Adding the new findings to the graph nodes

In [ ]:
FE.train.Hierarchy_cls.update_graph(hyperparameters=FE.hyperparameters)
FE.test. Hierarchy_cls.update_graph(hyperparameters=FE.hyperparameters)

### Measuring the updated metrics (predictions and losses, thresholds, aucs, etc.)

In [ ]:
params = {key: getattr(FE, key) for key in ['model', 'config', 'hyperparameters']}
FE.train = CalculateNewFindings.get_updated_data(data=FE.train, **params)
FE.test  = CalculateNewFindings.get_updated_data(data=FE.test , **params)

In [ ]:
FE.hyperparameters['ROC']

### Saving the metrics: AUC, threshold, accuracy

In [ ]:
FE.save_metrics()

In [ ]:
FE.get_metric(metric='AUC', data_mode='test')